In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import scipy.stats as stats
from sklearn.metrics import r2_score, accuracy_score, fbeta_score
from sklearn.model_selection import RandomizedSearchCV

df = pd.read_stata(r"D:\World Bank\Honduras PMT benchmark\Data_out\CONSOLIDADA_2023_clean.dta")

In [60]:
df["logingreso_tr"] = stats.mstats.winsorize(df["logingreso"], limits=(0.1, 0.1))

C:\Users\Nico\AppData\Local\Temp\ipykernel_10912\2180309289.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["logingreso_tr"] = stats.mstats.winsorize(df["logingreso"], limits=(0.1, 0.1))


In [58]:
# df["EDAD2"] = df["EDAD"]**2
# oh1 = pd.get_dummies(df['CIVIL'], prefix="CIVIL_", dummy_na=True)
# oh2 = pd.get_dummies(df['CH307'], prefix="dis_", dummy_na=True)
# oh3 = pd.get_dummies(df['CH308'], prefix="orig_", dummy_na=True)
# oh4 = pd.get_dummies(df['OC609'], prefix="trab1_", dummy_na=True)
# oh5 = pd.get_dummies(df['CATEGOP'], prefix="trab2_", dummy_na=True)
# oh6 = pd.get_dummies(df['RAMAOP'], prefix="trab3_", dummy_na=True)
# oh7 = pd.get_dummies(df['OCUPAOP'], prefix="trab4_", dummy_na=True)
# oh8 = pd.get_dummies(df['DOMI'], prefix="domi_", dummy_na=True)
# oh9 = pd.get_dummies(df['SEXO'], prefix="genero_", dummy_na=True)
# oh10 = pd.get_dummies(df['ED01'], prefix="ed_", dummy_na=True)
# oh11 = pd.get_dummies(df['CA501'], prefix="ca501_", dummy_na=True)
# oh12 = pd.get_dummies(df['UR'], prefix="ur_", dummy_na=True)
# oh13 = pd.get_dummies(df['NBI'], prefix="nbi_", dummy_na=True)
# one_hots = pd.concat([oh1, oh2, oh3, oh4, oh5, oh6, oh7, oh8, oh9, oh10, oh11, oh12, oh13], axis=1)
# one_hots_cols = one_hots.columns.to_list()
# df = pd.concat([df, one_hots], axis=1)
# df = pd.concat([df], axis=1)

In [202]:
# Variables globales para las distintas estimaciones **
vars_pmtoriginal = ["Ocupacion_bien","Paredes_bien","Pension_bien","Refri_mal","Remesas_bien","Aire_mal","Alquileres_bien","Alumbrado_bien","Basura_bien","Carro_mal","Cocina2_bien","Compu_mal","Dependencia","dv111","Ed_diversif_bien","Ed_univer_bien","edad_0_5","edad_15_21","edad_60_120","edad_6_14","Estufa_mal","Vivienda2_bien","EqSonido_mal","HaySanitario_bien","Sanitario_bien","Civil_mal","Cocina_bien","Cable_mal","Hacinamiento","Moto_mal","Piso_mal","Bici_mal","Exterior_bien","Dominio_1","Dominio_2","Dominio_3","Vivienda_bien","Agua2_bien","Agua_bien","TV_mal","Telefono_mal"]
IPM_vars = ["privacion_agua_h","privacion_saneamiento_h","privacion_cocina_h","privacion_educ_h","privacion_asistencia_h","privacion_alfab_h","privacion_elec_h","privacion_piso_h","privacion_techo_h","privacion_segsoc_h","privacion_desocup_h","privacion_subemp_h","privacion_ocup_h","privacion_trabinf_h","privacion_trabadol1_h","privacion_trabadol2_h","privacion_pared_h","privacion_hacina_h"]
vars_conjunto1 = vars_pmtoriginal + IPM_vars

#* Eliminamos las variables que eran muy multicolineales, eran falseables o no dependian de los hogares
vars_conjunto2 = ["privacion_agua_h","privacion_saneamiento_h","privacion_cocina_h","privacion_educ_h","privacion_asistencia_h","privacion_alfab_h","Piso_mal","privacion_techo_h","Paredes_bien","EqSonido_mal","HaySanitario_bien","Sanitario_bien","Civil_mal","Cocina_bien","Hacinamiento","Cable_mal","Moto_mal","Bici_mal","Exterior_bien","Dominio_1","Dominio_2","Dominio_3","Ocupacion_bien","Pension_bien","Refri_mal","Aire_mal","Carro_mal","Cocina2_bien","Compu_mal","Dependencia","dv111","Ed_diversif_bien","Ed_univer_bien","edad_0_5","edad_15_21","edad_60_120","edad_6_14","Estufa_mal"]

In [2]:
def fit_linear_model(df, y_var, vars):
    """ Fit a linear model to the data """
    
    from sklearn.linear_model import LinearRegression
    
    df_test = df[df.test_set==1]
    df_test = df_test[vars + [y_var, "FACTOR_P"]].dropna()
    df_train = df[df.test_set==0]
    df_train = df_train[vars + [y_var, "FACTOR_P"]].dropna()
    X_test = df_test[vars]
    y_test = df_test[y_var]
    w_test = df_test["FACTOR_P"]
    X_train = df_train[vars]
    y_train = df_train[y_var]
    w_train = df_train["FACTOR_P"]
    
    model = LinearRegression()
    model.fit(X_train, y_train, sample_weight=w_train)
    y_test_preds = model.predict(X_test)
    y_train_preds = model.predict(X_train)
    
    return model, y_test_preds, y_train_preds, y_test, y_train, w_test, w_train
    
    

def fit_xgboost_reg(df, y_var, vars, params_grid, scoring="neg_mean_absolute_error"):
    """
    Trains and evaluates an XGBoost regression model using a randomized grid search for hyperparameter tuning.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing the dataset with both training and test sets, including features and target variables.
        The DataFrame should have a column 'test_set' where 0 indicates training data and 1 indicates test data.
    
    y_var: str
        A column name representing the variable to use as labels of the model.

    vars : list
        A list of column names representing the independent variables (features) used for training the model.
        
    params_grid : dict
        Dictionary where keys are XGBoost hyperparameters and values are lists of possible values for those
        hyperparameters. This grid is used for randomized hyperparameter tuning.

    Returns:
    --------
    random_search : RandomizedSearchCV object
        The fitted RandomizedSearchCV object containing the best estimator and results from the grid search.
        
    Workflow:
    ---------
    1. Splits the input DataFrame into training and test sets based on the 'test_set' column.
    2. Prepares the feature matrix (X) and target variable (y) for both training and test sets.
    3. Converts the data into DMatrix format required for XGBoost.
    4. Initializes an XGBoost regressor with the 'hist' tree method and 'cuda' for GPU acceleration.
    5. Conducts a RandomizedSearchCV to tune hyperparameters based on a given parameter grid.
    6. Fits the model to the training data and evaluates its performance on both the training and test sets.
    7. Prints R-squared metrics for both training and test sets.
    8. Returns the fitted RandomizedSearchCV object.
    """

    df_test = df[df.test_set==1]
    df_train = df[df.test_set==0]
    X_test = df_test[vars]
    y_test = df_test[y_var]
    w_test = df_test["FACTOR_P"]
    X_train = df_train[vars]
    y_train = df_train[y_var]
    w_train = df_train["FACTOR_P"]
    
    model = xgb.XGBRegressor(
        tree_method="hist",
        device="cuda",
    )

    random_search = RandomizedSearchCV(
        model, 
        param_distributions=params_grid, 
        n_iter=500, 
        cv=5, 
        scoring='neg_mean_absolute_error', 
        verbose=0, 
    )

    random_search.fit(X_train, y_train)
    y_test_preds = random_search.best_estimator_.predict(X_test)
    y_train_preds = random_search.best_estimator_.predict(X_train)
    
    return random_search, y_test_preds, y_train_preds, y_test, y_train, w_test, w_train

def asigna_beneficios(y, weights, percentage):
    df = pd.DataFrame({"y": y, "weights": weights})
    df = df.sort_values("y")

    ranking = df["weights"].cumsum()
    ranking_pct = ranking / ranking.max()    
    y_bool = (ranking_pct < percentage)    
    y_bool = y_bool.sort_index()
    
    return y_bool

def compute_metrics(y, y_preds, weights=None, percentage=.667):
    r_test = r2_score(y, y_preds, sample_weight=weights, multioutput='uniform_average')
    print(f"R2 test: {r_test}")

    y_bool = (y.values < np.quantile(y_preds, percentage))
    y_preds_bool = (y_preds < np.quantile(y_preds, percentage))

    acc_test = accuracy_score(y_bool, y_preds_bool, sample_weight=weights)
    print(f"Accuracy Test: {acc_test}")
    
    f2_test = fbeta_score(y_bool, y_preds_bool, beta=2, sample_weight=weights)
    print(f"F2 Test: {f2_test}")
    
    return r_test, acc_test, f2_test

def compute_metrics_urru(y_ur, y_preds_ur, y_ru, y_preds_ru, weights_ur=None, weights_ru=None, percentage_ur=.667, percentage_ru=.667):

    # Create boolean arrays for each dataset
    y_bool_ru = asigna_beneficios(y_ru, weights_ru, percentage_ru)
    y_preds_bool_ru = asigna_beneficios(y_preds_ru, weights_ru, percentage_ru)
    y_bool_ur = asigna_beneficios(y_ur, weights_ur, percentage_ur)
    y_preds_bool_ur = asigna_beneficios(y_preds_ur, weights_ur, percentage_ur)

    # Create single arrays for all data
    y_bool = pd.concat([pd.Series(y_bool_ru), pd.Series(y_bool_ur)])
    y_preds_bool = pd.concat([pd.Series(y_preds_bool_ru), pd.Series(y_preds_bool_ur)])
    
    y = pd.concat([pd.Series(y_ru), pd.Series(y_ur)])
    y_preds = pd.concat([pd.Series(y_preds_ru), pd.Series(y_preds_ur)])
    weights = pd.concat([pd.Series(weights_ru), pd.Series(weights_ur)])

    r_test = r2_score(y, y_preds, sample_weight=weights, multioutput='uniform_average')
    print(f"R2 test: {r_test}")

    acc_test = accuracy_score(y_bool, y_preds_bool, sample_weight=weights)
    print(f"Accuracy Test: {acc_test}")
    
    f2_test = fbeta_score(y_bool, y_preds_bool, beta=2, sample_weight=weights)
    print(f"F2 Test: {f2_test}")
    
    return r_test, acc_test, f2_test


In [211]:
params_grid = {
    'learning_rate': stats.uniform(0.005, 0.1),
    'reg_alpha': stats.uniform(0, 0.1),
    'reg_lambda': stats.uniform(0.7, 0.3),
    'gamma':stats.uniform(0.1, 0.5),
    'max_depth': stats.randint(5, 30),
    'min_child_weight': stats.randint(5, 30),
    'n_estimators': stats.randint(50, 500),
    'colsample_bytree':stats.uniform(0.5, 0.5),
    'subsample': stats.uniform(0.5, .25),
}

casos = {
    "vars_all": {
        "df":df, 
        "vars": vars_conjunto1,
    },
    "vars_min": {
        "df":df, 
        "vars": vars_conjunto2,
    },

}

resultados = {}
for name, params in casos.items():
    print(f"Running model: {name}")
    df_model, vars = params.values()
    # # Nacional
    # results, y_test_preds, y_train_preds, y_test, y_train = fit_xgboost_reg(df_model, "logingreso", vars, params_grid)
    # r2_test, acc_test = compute_metrics(y_test, y_test_preds)
    # resultados[name + "_reg_nac"] = {}
    # resultados[name + "_reg_nac"]["df"] = df_model
    # resultados[name + "_reg_nac"]["results"] = results
    # resultados[name + "_reg_nac"]["best_params"] = results.best_params_
    # resultados[name + "_reg_nac"]["r_test"]  = r2_test
    # resultados[name + "_reg_nac"]["y_test_preds"] = y_test_preds

    # Modelo lineal (para garantizar que esté todo ok)
    model_ru, y_test_preds_ru, y_train_preds_ru, y_test_ru, y_train_ru, w_test_ru, w_train_ru = fit_linear_model(df_model[df_model["UR"]=="Rural"], "logingreso", vars)
    model_ur, y_test_preds_ur, y_train_preds_ur, y_test_ur, y_train_ur, w_test_ur, w_train_ur = fit_linear_model(df_model[df_model["UR"]=="Urbana"], "logingreso", vars)
    print("Linear Model")
    r2_test, acc_test, f2_test = compute_metrics_urru(y_test_ur, y_test_preds_ur, y_test_ru, y_test_preds_ru, weights_ur=w_test_ur, weights_ru=w_test_ru)
    resultados[name + "_linreg_urru"] = {}
    resultados[name + "_linreg_urru"]["df"] = df_model
    resultados[name + "_linreg_urru"]["model"] = (model_ru, model_ur)
    resultados[name + "_linreg_urru"]["best_params"] = {"Rural": (model_ru.intercept_, model_ru.coef_), "Urbana": (model_ur.intercept_, model_ur.coef_)}
    resultados[name + "_linreg_urru"]["r_test"]  = r2_test
    resultados[name + "_linreg_urru"]["f2_test"] = f2_test
    
    # Urbano y Rural estimados por separado
    model_ru, y_test_preds_ru, y_train_preds_ru, y_test_ru, y_train_ru, w_test_ru, w_train_ru = fit_xgboost_reg(df_model[df_model["UR"]=="Rural"], "logingreso", vars, params_grid)
    model_ur, y_test_preds_ur, y_train_preds_ur, y_test_ur, y_train_ur, w_test_ur, w_train_ur = fit_xgboost_reg(df_model[df_model["UR"]=="Urbana"], "logingreso", vars, params_grid)
    print("XGBoost Model")
    r2_test, acc_test, f2_test = compute_metrics_urru(y_test_ur, y_test_preds_ur, y_test_ru, y_test_preds_ru, weights_ur=w_test_ur, weights_ru=w_test_ru)
    resultados[name + "_xgboost_urru"] = {}
    resultados[name + "_xgboost_urru"]["df"] = df_model
    resultados[name + "_xgboost_urru"]["model"] = (model_ru, model_ur)
    resultados[name + "_xgboost_urru"]["best_params"] = {"Rural": model_ru.best_params_, "Urban": model_ur.best_params_}
    resultados[name + "_xgboost_urru"]["r_test"]  = r2_test
    resultados[name + "_xgboost_urru"]["f2_test"] = f2_test


    # results, y_test_preds, r_test, r_train, cm = fit_xgboost_cla(df_model, "pobreza", vars, params_grid)
    # resultados[name + "_cla"] = {}
    # resultados[name + "_cla"]["df"] = df_model
    # resultados[name + "_cla"]["results"] = results
    # resultados[name + "_cla"]["best_params"] = results.best_params_
    # resultados[name + "_cla"]["r_test"]  = r_test
    # resultados[name + "_cla"]["r_train"] = r_train
    # resultados[name + "_cla"]["confusion_matrix"] = cm
    # resultados[name + "_cla"]["y_test_preds"] = y_test_preds


Running model: vars_all
Linear Model
R2 test: 0.3756539821624756
Accuracy Test: 0.7775374293607445
F2 Test: 0.8332333456739488
XGBoost Model
R2 test: 0.39656704664230347
Accuracy Test: 0.7802456465119997
F2 Test: 0.8350833638456354
Running model: vars_min
Linear Model
R2 test: 0.3561222553253174
Accuracy Test: 0.7623269579811418
F2 Test: 0.8217639048958617


C:\Users\Nico\AppData\Roaming\Python\Python312\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


XGBoost Model
R2 test: 0.3645418882369995
Accuracy Test: 0.765901087989646
F2 Test: 0.8242015932714988


In [11]:
(df_test["pobre_ingreso"]*df_test["FACTOR_P"]).sum() / df_test["FACTOR_P"].sum()

0.66671354

In [14]:
pd.crosstab(df_test["pobre_ingreso"], df_test["pobreza"])

pobreza,0.0,1.0
pobre_ingreso,,
False,1544,280
True,386,2699


In [13]:
df_test

,HOGAR,NPER,DOMINIO,DEPMUESTRA,COR_PRE,NUM_REC,NUM_HOG,V01,V02,V02OTRO,V03,V03OTRO,V04,V04OTRO,V05,V05OTRA,V06,V07,V07OTRO,V08,V08OTRA,V09,V10,V11,V12,H01_1,H01_2,H01_3,H01_4,H01_5,H01_6,H01_7,H01_8,H01_9,H01_10,H01_11,H01_12,H02,H03,H04,H04OTRO,H05,H06,H07,H07OTRO,H08,H09,SEN01,SEN02,SEN03,SEN03_OTROS,SEN04,ME01,ME01HUSA,ME01MUSA,ME01HESP,ME01MESP,ME01HMEX,ME01MMEX,ME01HCA,ME01MCA,ME01HOTRO,ME01MOTRO,ME02,RELA_J,SEXO,EDAD,CIVIL,CH303,CH304,CH305,CH306,CH307,CH308,ED01,ED02,ED03,ED04,ED04ESP,ED05,ED06,ED07,ED08,ED09,ED10,ED11,ED11A,ED11B,ED12,ED13,ED14,ED15,ED16,ED17,ED18,ED19,ED20,ED20ESP,CD01,CD02_DEPT,CD02_MUNI,CD02_ALDEA,CD02_PAIS,CD03,CD03_2_TIEMPO,CD04_DEPT,CD04_MUNI,CD04_ALDEA,CD04_PAIS,CD05,CA501,CA502,CA503,CA504,CA505,CA505ESP,CA506,CA507,CA508,CA508A,CA509,CA509A,CA509B,CA510,CA511,CA512,CA513,CA514,CP514_OTRA,CA515,CA_515_OTRO,CA516TIEMPO,CA516DSM,CA517,CA518,CA518TIEMPO,CA519,CA520,CA521,CA522,CA523,CA524,CA525,OC_605_LUNES,OC_605_MARTES,OC_605_MIERCOLES,OC_605_JUEVES,OC_605_VIERNES,OC_605_SABADO,OC_605_DOMINGO,OC606,OC607,OC608,OC_608_CUANTAS,OC609,OC610,OC611,OC612,OC613,OC614,OC615,OC616,OC617,OC617_TIPO_DE_MONEDA,OC618,OC619_ALIMENTOS,OC619_HABITACION,OC619_ROPA_CALZADO,OC619_TRANSPORTE,OC619_COMISION,OC619_BONIFICACION,OC619_PROPINAS,OC619_HORAS_EXTRAS,OC620,OC621,OC622,OC623,OC624,OC625,OC626,OC627,OC628,OC629,OC630,OC631,OC632,OC633,OC634,OC635,OC636,OC637,OC638,OC639,OC_605_LUNES1,OC_605_MARTES1,OC_605_MIERCOLES1,OC_605_JUEVES1,OC_605_VIERNES1,OC_605_SABADO1,OC_605_DOMINGO1,OC6061,OC6071,OC6081,OC_608_CUANTAS1,OC6091,OC6101,OC6111,OC6121,OC6131,OC6141,OC6151,OC6161,OC6171,OC617_TIPO_DE_MONEDA1,OC6181,OC619_ALIMENTOS1,OC619_HABITACION1,OC619_ROPA_CALZADO1,OC619_TRANSPORTE1,OC619_COMISION1,OC619_BONIFICACION1,OC619_PROPINAS1,OC619_HORAS_EXTRAS1,OC6201,OC6211,OC6221,OC6231,OC6241,OC6251,OC6261,OC6271,OC6281,OC6291,OC6301,OC6311,OC6321,OC6331,OC6341,OC6351,OC6361,OC6371,OC6381,OC6391,OIH01_LPS,OIH01_US,OIH02_LPS,OIH02_US,OIH03_LPS,OIH03_US,OIH04,OIH05_LPS,OIH05_US,OIH06_LPS,OIH06_US,OIH06_LPS_ESP,OIH06_US_ESP,OIH07_LPS,OIH07_US,OIH07_LPS_ESP,OIH07_US_ESP,OIH08,OIH09,OIH10,OIH11,OIH12_LPS,OIH12_US,OIH12_LPS_ESP,OIH12_US_ESP,OIH13,OIH14,OIH15,OIH16,OIH17,OIH18,OIH19_LPS,OIH19_US,OIH19_LPS_ESP,OIH19_US_ESP,OIH20_LPS,OIH20_US,OIH20_LPS_ESP,OIH20_US_ESP,OI002,OI002_1,OI002_2,OI002_3,OI002_4,OI002_5,OI002_6,OI002_7,OI002_8,OI002_9,OI002_10,OI002_11,OI002_12,OI002_OTROS,OI003,OI004,YTRAOP,CAMBIO,UR,YSMOP,YSEOP,YCMOP,YCEOP,YMOP,CATEGOP,YTRAB,YTRAOS,YSMOS,YSEOS,YCMOS,YCEOS,YMOS,YTOTHG,YPERHG,QUINTIL,POBREZA,POBREZA_PER,CONDACT,DOMI,ASAL,NIVEL,ANOSEST,ESTRAB,O01_CODIGO,O02_CODIGO,RAMAOP,OCUPAOP,SALMIN,NSALMIN,NUMSALMIN,ACTIVIDA,YHPENS,YHJUBI,YHALQU,YHOTROS,YHAYUF,YHAYUP,YHREME,YHEDUC,YHBONO,TOTPER,YTRABHG,YOTRFHG,YMOPHG,QUINTILH,FACTOR,REDAD1,TIPO_HOG,HACINA,NBI,logingreso,pobreza,no_pobreza,pobreza_ext,no_pobreza_ext,Dominio_1,Dominio_2,Dominio_3,Dominio_4,Vivienda_bien,Paredes_bien,Piso_mal,Agua_bien,Agua2_bien,Alumbrado_bien,Basura_bien,Vivienda2_bien,Hacinamiento,Cocina_bien,Cocina2_bien,Estufa_bien,HaySanitario_bien,Sanitario_bien,Refri_mal,Estufa_mal,TV_mal,Cable_mal,Radio_mal,EqSonido_mal,Telefono_mal,Carro_mal,Moto_mal,Bici_mal,Compu_mal,Aire_mal,Exterior_bien,Civil_mal,Ed_basica_bien,Ed_diversif_bien,Ed_univer_bien,Compu_bien,Trabajo_bien,Ocupacion_bien,edad,edad_jefe2,edad_0_5,edad_6_14,edad_15_21,edad_22_60,edad_60_120,dv111,dv112,Dependencia,hh_OIH01_LPS,hh_OIH01_US,hh_OIH02_LPS,hh_OIH05_LPS,hh_OIH05_US,pen_jub_monto,Pension_bien,hh_OIH03_LPS,Alquileres_bien,hh_OIH12_LPS,hh_OIH12_US,hh_OIH12_LPS_ESP,hh_OIH12_US_ESP,remesas_monto,Remesas_bien,privacion_agua,privacion_agua_h,privacion_saneamiento,privacion_saneamiento_h,privacion_cocina,privacion_cocina_h,privacion_educ,privacion_educ_h,privacion_asistencia,privacion_asistencia_h,privacion_alfab,privacion_alfab_h,horas,cotiza,privacion_segsoc,privacion_segsoc_h,privacion_desocup,privacion_desocup_h,privacion_subemp,privacion_subemp_h,privacion_ocup,q_no_ocup,p

In [42]:
# Asigna beneficios según Indice de Pobreza Multidimensional
pd.set_option('display.max_columns', None)
df["indice_pobreza_multi_neg"] = -df["indice_pobreza_multi"]

df_test = df[df.test_set==1]
r2_test = r2_score(df_test["logingreso"], df_test["indice_pobreza_multi"], sample_weight=df_test.FACTOR_P)
df_test_ur = df_test[df_test.UR=="Urbana"]
df_test_ru = df_test[df_test.UR=="Rural"]

df_test_ur["asignado_IPM"] = asigna_beneficios(df_test_ur.indice_pobreza_multi_neg, df_test_ur.FACTOR_P, 0.667)
df_test_ur["pobre_ingreso"] = asigna_beneficios(df_test_ur.logingreso, df_test_ur.FACTOR_P, 0.667)
df_test_ru["asignado_IPM"] = asigna_beneficios(df_test_ru.indice_pobreza_multi_neg, df_test_ru.FACTOR_P, 0.667)
df_test_ru["pobre_ingreso"] = asigna_beneficios(df_test_ru.logingreso, df_test_ru.FACTOR_P, 0.667)
print(pd.crosstab(df_test_ru["pobre_ingreso"], df_test_ru["asignado_IPM"]))

df_test = pd.concat([df_test_ur, df_test_ru])
acc_test = accuracy_score(df_test["pobreza"], df_test["asignado_IPM"], sample_weight=df_test.FACTOR_P)
f2_test = fbeta_score(df_test["pobreza"], df_test["asignado_IPM"], beta=2, sample_weight=df_test.FACTOR_P)
print(f"R2 test: {r2_test}")
print(f"Accuracy Test: {acc_test}")
print(f"F2 Test: {f2_test}")


asignado_IPM   False  True 
pobre_ingreso              
False            464    433
True             423   1091
R2 test: -0.5386184628874129
Accuracy Test: 0.6742223000886426
F2 Test: 0.7569472591422359


C:\Users\Nico\AppData\Local\Temp\ipykernel_13644\1911421648.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test_ur["asignado_IPM"] = asigna_beneficios(df_test_ur.indice_pobreza_multi_neg, df_test_ur.FACTOR_P, 0.667)
C:\Users\Nico\AppData\Local\Temp\ipykernel_13644\1911421648.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_ur["asignado_IPM"] = asigna_beneficios(df_test_ur.indice_pobreza_multi_neg, df_test_ur.FACTOR_P, 0.667)
C:\Users\Nico\AppData\Local\Temp\ipykernel_13644\1911421648.py:11: Perfor

In [62]:
df_test[["logingreso", "indice_pobreza_multi"]].corr()**2

,logingreso,indice_pobreza_multi
logingreso,1.000000,0.189002
indice_pobreza_multi,0.189002,1.000000


In [57]:
import statsmodels.api as sm
sm.OLS(df_test["logingreso"], sm.add_constant(df_test["indice_pobreza_multi_neg"])).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             logingreso   R-squared:                       0.189
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     1144.
Date:                Wed, 30 Oct 2024   Prob (F-statistic):          1.58e-225
Time:                        17:23:16   Log-Likelihood:                -6850.5
No. Observations:                4909   AIC:                         1.371e+04
Df Residuals:                    4907   BIC:                         1.372e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        8.6005      0.025    338.384      0.000       8.551       8.650
indice_pobreza_multi_neg     3.4426      0.102     33.817      0.000       3.243       3.642
==============================================================================
Omnibus:                     1399.680   Durbin-Watson:                   1.749
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8739.528
Skew:                          -1.211   Prob(JB):                         0.00
Kurtosis:                       9.071   Cond. No.                         7.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

-0.5387050705722647

In [38]:
df_test

,HOGAR,NPER,DOMINIO,DEPMUESTRA,COR_PRE,NUM_REC,NUM_HOG,V01,V02,V02OTRO,V03,V03OTRO,V04,V04OTRO,V05,V05OTRA,V06,V07,V07OTRO,V08,V08OTRA,V09,V10,V11,V12,H01_1,H01_2,H01_3,H01_4,H01_5,H01_6,H01_7,H01_8,H01_9,H01_10,H01_11,H01_12,H02,H03,H04,H04OTRO,H05,H06,H07,H07OTRO,H08,H09,SEN01,SEN02,SEN03,SEN03_OTROS,SEN04,ME01,ME01HUSA,ME01MUSA,ME01HESP,ME01MESP,ME01HMEX,ME01MMEX,ME01HCA,ME01MCA,ME01HOTRO,ME01MOTRO,ME02,RELA_J,SEXO,EDAD,CIVIL,CH303,CH304,CH305,CH306,CH307,CH308,ED01,ED02,ED03,ED04,ED04ESP,ED05,ED06,ED07,ED08,ED09,ED10,ED11,ED11A,ED11B,ED12,ED13,ED14,ED15,ED16,ED17,ED18,ED19,ED20,ED20ESP,CD01,CD02_DEPT,CD02_MUNI,CD02_ALDEA,CD02_PAIS,CD03,CD03_2_TIEMPO,CD04_DEPT,CD04_MUNI,CD04_ALDEA,CD04_PAIS,CD05,CA501,CA502,CA503,CA504,CA505,CA505ESP,CA506,CA507,CA508,CA508A,CA509,CA509A,CA509B,CA510,CA511,CA512,CA513,CA514,CP514_OTRA,CA515,CA_515_OTRO,CA516TIEMPO,CA516DSM,CA517,CA518,CA518TIEMPO,CA519,CA520,CA521,CA522,CA523,CA524,CA525,OC_605_LUNES,OC_605_MARTES,OC_605_MIERCOLES,OC_605_JUEVES,OC_605_VIERNES,OC_605_SABADO,OC_605_DOMINGO,OC606,OC607,OC608,OC_608_CUANTAS,OC609,OC610,OC611,OC612,OC613,OC614,OC615,OC616,OC617,OC617_TIPO_DE_MONEDA,OC618,OC619_ALIMENTOS,OC619_HABITACION,OC619_ROPA_CALZADO,OC619_TRANSPORTE,OC619_COMISION,OC619_BONIFICACION,OC619_PROPINAS,OC619_HORAS_EXTRAS,OC620,OC621,OC622,OC623,OC624,OC625,OC626,OC627,OC628,OC629,OC630,OC631,OC632,OC633,OC634,OC635,OC636,OC637,OC638,OC639,OC_605_LUNES1,OC_605_MARTES1,OC_605_MIERCOLES1,OC_605_JUEVES1,OC_605_VIERNES1,OC_605_SABADO1,OC_605_DOMINGO1,OC6061,OC6071,OC6081,OC_608_CUANTAS1,OC6091,OC6101,OC6111,OC6121,OC6131,OC6141,OC6151,OC6161,OC6171,OC617_TIPO_DE_MONEDA1,OC6181,OC619_ALIMENTOS1,OC619_HABITACION1,OC619_ROPA_CALZADO1,OC619_TRANSPORTE1,OC619_COMISION1,OC619_BONIFICACION1,OC619_PROPINAS1,OC619_HORAS_EXTRAS1,OC6201,OC6211,OC6221,OC6231,OC6241,OC6251,OC6261,OC6271,OC6281,OC6291,OC6301,OC6311,OC6321,OC6331,OC6341,OC6351,OC6361,OC6371,OC6381,OC6391,OIH01_LPS,OIH01_US,OIH02_LPS,OIH02_US,OIH03_LPS,OIH03_US,OIH04,OIH05_LPS,OIH05_US,OIH06_LPS,OIH06_US,OIH06_LPS_ESP,OIH06_US_ESP,OIH07_LPS,OIH07_US,OIH07_LPS_ESP,OIH07_US_ESP,OIH08,OIH09,OIH10,OIH11,OIH12_LPS,OIH12_US,OIH12_LPS_ESP,OIH12_US_ESP,OIH13,OIH14,OIH15,OIH16,OIH17,OIH18,OIH19_LPS,OIH19_US,OIH19_LPS_ESP,OIH19_US_ESP,OIH20_LPS,OIH20_US,OIH20_LPS_ESP,OIH20_US_ESP,OI002,OI002_1,OI002_2,OI002_3,OI002_4,OI002_5,OI002_6,OI002_7,OI002_8,OI002_9,OI002_10,OI002_11,OI002_12,OI002_OTROS,OI003,OI004,YTRAOP,CAMBIO,UR,YSMOP,YSEOP,YCMOP,YCEOP,YMOP,CATEGOP,YTRAB,YTRAOS,YSMOS,YSEOS,YCMOS,YCEOS,YMOS,YTOTHG,YPERHG,QUINTIL,POBREZA,POBREZA_PER,CONDACT,DOMI,ASAL,NIVEL,ANOSEST,ESTRAB,O01_CODIGO,O02_CODIGO,RAMAOP,OCUPAOP,SALMIN,NSALMIN,NUMSALMIN,ACTIVIDA,YHPENS,YHJUBI,YHALQU,YHOTROS,YHAYUF,YHAYUP,YHREME,YHEDUC,YHBONO,TOTPER,YTRABHG,YOTRFHG,YMOPHG,QUINTILH,FACTOR,REDAD1,TIPO_HOG,HACINA,NBI,logingreso,pobreza,no_pobreza,pobreza_ext,no_pobreza_ext,Dominio_1,Dominio_2,Dominio_3,Dominio_4,Vivienda_bien,Paredes_bien,Piso_mal,Agua_bien,Agua2_bien,Alumbrado_bien,Basura_bien,Vivienda2_bien,Hacinamiento,Cocina_bien,Cocina2_bien,Estufa_bien,HaySanitario_bien,Sanitario_bien,Refri_mal,Estufa_mal,TV_mal,Cable_mal,Radio_mal,EqSonido_mal,Telefono_mal,Carro_mal,Moto_mal,Bici_mal,Compu_mal,Aire_mal,Exterior_bien,Civil_mal,Ed_basica_bien,Ed_diversif_bien,Ed_univer_bien,Compu_bien,Trabajo_bien,Ocupacion_bien,edad,edad_jefe2,edad_0_5,edad_6_14,edad_15_21,edad_22_60,edad_60_120,dv111,dv112,Dependencia,hh_OIH01_LPS,hh_OIH01_US,hh_OIH02_LPS,hh_OIH05_LPS,hh_OIH05_US,pen_jub_monto,Pension_bien,hh_OIH03_LPS,Alquileres_bien,hh_OIH12_LPS,hh_OIH12_US,hh_OIH12_LPS_ESP,hh_OIH12_US_ESP,remesas_monto,Remesas_bien,privacion_agua,privacion_agua_h,privacion_saneamiento,privacion_saneamiento_h,privacion_cocina,privacion_cocina_h,privacion_educ,privacion_educ_h,privacion_asistencia,privacion_asistencia_h,privacion_alfab,privacion_alfab_h,horas,cotiza,privacion_segsoc,privacion_segsoc_h,privacion_desocup,privacion_desocup_h,privacion_subemp,privacion_subemp_h,privacion_ocup,q_no_ocup,p

C:\Users\Nico\AppData\Local\Temp\ipykernel_13644\315020269.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["indice_pobreza_multi_neg"] = -df["indice_pobreza_multi"]


In [29]:
pd.crosstab(df_test["asignado_PMT"], df_test["asignado_IPM"])

KeyError: 'asignado_PMT'

In [9]:
from sklearn.metrics import fbeta_score, accuracy_score

scores = {}

df_test = df[df.test_set==1]
df_test["asignado_test"] = (df["YPERHG"] > df["YPERHG"].quantile(0.6127813))

# IPM
scores["IPM"] = {}
accuracy = accuracy_score(y_true=df_test["asignado_test"], y_pred=df_test["pobreza_multidim"])
f2 = fbeta_score(y_true=df_test["asignado_test"], y_pred=df_test["pobreza_multidim"], beta=2)
scores["IPM"]["accuracy_score"] = accuracy
scores["IPM"]["f2"] = f2

# PMT Carlos
scores["PMT_carlos"] = {}
threshold = np.quantile(df_test["log_ingreso_est_PMT_carlos"].values, 0.6127813)
PMT_classification = (df_test["log_ingreso_est_PMT_carlos"].values < threshold)

accuracy = accuracy_score(y_true=df_test["asignado_test"].values, y_pred=PMT_classification)
f2 = fbeta_score(y_true=df_test["asignado_test"].values, y_pred=PMT_classification, beta=2)
scores["PMT_carlos"]["accuracy_score"] = accuracy
scores["PMT_carlos"]["f2"] = f2


for name, caso in resultados.items():
    print(name)
    scores[name] = {}
    df_caso = casos[name[:-4]]["df"]
    y_test = df_caso[df_caso.test_set == 1].logingreso
    
    threshold = np.quantile(y_test.values, 0.6127813)
    real_classification = (y_test.values < threshold)

    y_test_preds = caso["y_test_preds"]
    if y_test_preds.dtype == "float32":
        threshold = np.quantile(y_test_preds, 0.6127813)
        preds_classification = (y_test_preds < threshold)

    accuracy = accuracy_score(y_true=real_classification, y_pred=preds_classification)
    f2 = fbeta_score(y_true=real_classification, y_pred=preds_classification, beta=2)
    scores[name]["accuracy_score"] = accuracy
    scores[name]["f2"] = f2


df = pd.DataFrame().from_dict(scores, orient="index")
df.to_excel(r"D:\World Bank\Honduras PMT benchmark\Outputs\resultados modelos boosting.xlsx")

C:\Users\Nico\AppData\Local\Temp\ipykernel_2356\2088539463.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["asignado_test"] = (df["YPERHG"] > df["YPERHG"].quantile(0.6127813))


vars_all_outliers_min_reg
vars_all_outliers_min_cla
vars_all_outliers_all_reg
vars_all_outliers_all_cla
vars_all_outliers_30_reg
vars_all_outliers_30_cla
vars_allPMT_outliers_all_reg
vars_allPMT_outliers_all_cla
vars_allPMT_outliers_30_reg
vars_allPMT_outliers_30_cla
vars_allPMT-IPM_outliers_all_reg
vars_allPMT-IPM_outliers_all_cla
vars_allPMT-IPM_outliers_30_reg
vars_allPMT-IPM_outliers_30_cla
vars_allPMTmin_outliers_all_reg
vars_allPMTmin_outliers_all_cla
vars_allPMTmin_outliers_30_reg
vars_allPMTmin_outliers_30_cla
